In [1]:
import pandas as pd
import numpy as np
%matplotlib inline

# 2. 予測（介護記録と加速度センサーを使用）

## 2.1 前処理

In [2]:
import numpy as np
import pandas as pd
import datetime

import my_module

**使い勝手を考え別のデータフレームにします。必須ではありません**。

In [3]:
user_df, df_care_train, df_care_test = my_module.read_all_data()
act_df = pd.concat([df_care_train, df_care_test])

In [4]:
user_df.head() 

,subject_id,datetime,x,y,z
0,8,2018-06-03T07:55:29.965+0900,2.375,-11.837,-0.664
1,8,2018-06-03T08:29:26.250+0900,2.375,3.486,8.968
2,8,2018-06-03T09:22:56.925+0900,2.375,4.405,8.436
3,8,2018-06-02T13:01:17.956+0900,2.375,4.367,8.849
4,8,2018-06-03T09:28:48.797+0900,2.375,4.060,8.707


In [5]:
act_df.head() 

,id,user_id,activity_type_id,activity_type,target_id,activity2user_id,start,finish,year-month-date-hour
0,4173,15,26,休憩,15,25397,2018-03-31 21:29:28+09:00,2018-03-31 22:02:45+09:00,2018-03-31-21.0
1,4410,15,26,休憩,15,26067,2018-04-03 21:49:23+09:00,2018-04-03 22:13:44+09:00,2018-04-03-21.0
2,6280,15,26,休憩,15,31067,2018-04-15 21:53:08+09:00,2018-04-15 22:14:46+09:00,2018-04-15-21.0
3,7928,15,26,休憩,15,34782,2018-04-21 21:34:30+09:00,2018-04-21 22:02:20+09:00,2018-04-21-21.0
4,12797,15,26,休憩,15,45514,2018-05-06 21:49:48+09:00,2018-05-06 22:08:50+09:00,2018-05-06-21.0


In [6]:
user_df = user_df.dropna() # to drop the nan values
user_df = user_df.sort_values('datetime') # While doing time series analysis it is very important to sort the file according to time stamp
user_df = user_df.drop_duplicates() # to drop duplicate columns

In [7]:
act_df = act_df.drop_duplicates('id') # to drop duplicate columns
act_df = act_df.dropna() # to drop the nan values

**Now we need to format the time stamp**

In [8]:
# Convert [datetime/start/finish] to datetime
user_df['datetime']= pd.to_datetime(user_df['datetime'], format='%Y-%m-%dT%H:%M:%S')
act_df['start'] = pd.to_datetime(act_df['start'], format='%Y-%m-%d %H:%M:%S')
act_df['finish'] = pd.to_datetime(act_df['finish'], format='%Y-%m-%d %H:%M:%S')

In [9]:
act_df['year-month-date'] = act_df['start'].dt.date.astype(str) # to create year-month-date column
act_df['hour'] = act_df['start'].dt.hour # to create hour column
act_df['year-month-date-hour'] = act_df['year-month-date'].astype(str) + '-' +act_df['hour'].astype(str).str.zfill(2) # to create year-month-date-hour column

act_df = act_df.sort_values('start') # to sort values

ACT_LABELS = list(act_df['activity_type_id'].value_counts().index) # 1)Activity Type List
USERS = list(act_df['user_id'].value_counts().index) # 2) User ID List

In [10]:
act_df.head()

,id,user_id,activity_type_id,activity_type,target_id,activity2user_id,start,finish,year-month-date-hour,year-month-date,hour
3371,179,14,18,申し送り・ミーティング,3,14624,2018-02-02 09:47:16+09:00,2018-02-02 09:47:21+09:00,2018-02-02-09,2018-02-02,9
3294,187,14,10,日中利用者対応,35,12354,2018-02-02 11:12:03+09:00,2018-02-02 11:09:40+09:00,2018-02-02-11,2018-02-02,11
711,189,14,17,手書き記録,30,11802,2018-02-02 11:27:59+09:00,2018-02-02 11:28:00+09:00,2018-02-02-11,2018-02-02,11
817,192,14,2,食事・服薬,30,11179,2018-02-02 11:56:46+09:00,2018-02-02 12:16:47+09:00,2018-02-02-11,2018-02-02,11
823,193,14,2,食事・服薬,32,11855,2018-02-02 12:14:54+09:00,2018-02-02 12:14:55+09:00,2018-02-02-12,2018-02-02,12


**ここで、午前0時から午後23時までの各時間帯のデータ数を確認してみましょう。**

In [11]:
user_df['datetime'].dt.hour.value_counts().sort_index()

0     380884
1     424456
2     378617
3     324604
4     346341
5     353996
6     431816
7     419731
8     394879
9     324724
10    309995
11    251515
12    261091
13    284904
14    245093
15    251276
16    256584
17    422510
18    359905
19    431961
20    393386
21    383916
22    390492
23    359618
Name: datetime, dtype: int64

In [12]:
print('Activity Type IDs List', ACT_LABELS) # to see which activities are done by this user.
display(act_df.head())

Activity Type IDs List [12, 4, 2, 10, 11, 6, 19, 1, 9, 18, 3, 16, 13, 20, 17, 24, 22, 23, 26, 14, 5, 7, 8, 25, 15, 27, 28, 21]


,id,user_id,activity_type_id,activity_type,target_id,activity2user_id,start,finish,year-month-date-hour,year-month-date,hour
3371,179,14,18,申し送り・ミーティング,3,14624,2018-02-02 09:47:16+09:00,2018-02-02 09:47:21+09:00,2018-02-02-09,2018-02-02,9
3294,187,14,10,日中利用者対応,35,12354,2018-02-02 11:12:03+09:00,2018-02-02 11:09:40+09:00,2018-02-02-11,2018-02-02,11
711,189,14,17,手書き記録,30,11802,2018-02-02 11:27:59+09:00,2018-02-02 11:28:00+09:00,2018-02-02-11,2018-02-02,11
817,192,14,2,食事・服薬,30,11179,2018-02-02 11:56:46+09:00,2018-02-02 12:16:47+09:00,2018-02-02-11,2018-02-02,11
823,193,14,2,食事・服薬,32,11855,2018-02-02 12:14:54+09:00,2018-02-02 12:14:55+09:00,2018-02-02-12,2018-02-02,12


In [13]:
# これは、両ファイルのUSERSを一致させるためです。
SUBJECTS = user_df['subject_id'].unique()
df_label_user = act_df[act_df['user_id'].isin(SUBJECTS)]

In [14]:
df_label_user = df_label_user.reset_index()
df_label_user = df_label_user.drop(columns='index')

**タイムスタンプでデータをセグメント化します。**  
**各セグメントウィンドウは、ケアレコードファイルのアクティビティの開始と終了の時間によって抽出されます**。

In [15]:
df_label_user.head()

,id,user_id,activity_type_id,activity_type,target_id,activity2user_id,start,finish,year-month-date-hour,year-month-date,hour
0,179,14,18,申し送り・ミーティング,3,14624,2018-02-02 09:47:16+09:00,2018-02-02 09:47:21+09:00,2018-02-02-09,2018-02-02,9
1,187,14,10,日中利用者対応,35,12354,2018-02-02 11:12:03+09:00,2018-02-02 11:09:40+09:00,2018-02-02-11,2018-02-02,11
2,189,14,17,手書き記録,30,11802,2018-02-02 11:27:59+09:00,2018-02-02 11:28:00+09:00,2018-02-02-11,2018-02-02,11
3,192,14,2,食事・服薬,30,11179,2018-02-02 11:56:46+09:00,2018-02-02 12:16:47+09:00,2018-02-02-11,2018-02-02,11
4,193,14,2,食事・服薬,32,11855,2018-02-02 12:14:54+09:00,2018-02-02 12:14:55+09:00,2018-02-02-12,2018-02-02,12


### 加速度センサーのデータを1サンプルごとに分割（＝1回分のアクティビティ）

In [16]:
"""
Make Segment.
A segment is a group of continuous sensor data for which feature calculation (statistic calculation) is performed in human activity recognition using sensor data.
"""
seg_label_list = [] # segment's (1 sample's) label list (label mean "activity_type_id", "user_id" etc.).
seg_list = [] # segment's (1 sample's) accelerometer data list.

for i in list(df_label_user.index):
    started_at = df_label_user.loc[i, 'start']
    finished_at = df_label_user.loc[i, 'finish']
    seg = user_df[(user_df["datetime"] >=started_at) & (user_df["datetime"] <= finished_at)]
    seg_label = df_label_user.loc[i, "activity_type_id"]
    if (len(seg)!=0):
        seg_list.append(seg)
        seg_label_list.append(seg_label)


## 2.2  特徴量の作成

### ・ 介護記録
まず、介護記録から時間の特徴を抽出します

In [17]:
date_start = act_df['start'].min()
date_finish = act_df['start'].max()

In [18]:
"""
開始日から終了日までの1時間単位のデータフレームを作成します。
曜日(weekday)は特徴量として使用されます。
"""
import datetime

time = date_start.date()
time_list = []
while time <= date_finish:
    for hour in range(0, 24):
        year_month_date_hour = str(time) + '-' +str(hour).zfill(2)
        time_list.append([year_month_date_hour, hour, time, time.weekday()])
    time = time + datetime.timedelta(days=1)

cols=['year-month-date-hour', 'hour', 'start', 'weekday']
df_y = pd.DataFrame(time_list, columns=cols)
del time_list, time

/tmp/ipykernel_740173/398634655.py:9: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  while time <= date_finish:


In [19]:
df_y.head()

,year-month-date-hour,hour,start,weekday
0,2018-02-02-00,0,2018-02-02,4
1,2018-02-02-01,1,2018-02-02,4
2,2018-02-02-02,2,2018-02-02,4
3,2018-02-02-03,3,2018-02-02,4
4,2018-02-02-04,4,2018-02-02,4


In [20]:
def makeFeatures(df):
    feat = df.groupby(['activity_type_id', 'year-month-date-hour'], as_index=False).count()[['activity_type_id', 'year-month-date-hour','id']].rename(columns={'id':'count'})
    X = df_y.copy()
    print(feat)
    acts = list(df['activity_type_id'].value_counts().index)
    for act in acts:
        X = pd.merge(X, feat[feat['activity_type_id'] == act][['year-month-date-hour', 'count']], on='year-month-date-hour', how="left").rename(columns={'count':str(act)})
    return X

tmp_df = act_df.copy()
features_cr_df = makeFeatures(tmp_df)
features_cr_df = features_cr_df.fillna(0).sort_values('year-month-date-hour')
features_cr_df = features_cr_df.reset_index()


      activity_type_id year-month-date-hour  count
0                    1        2018-03-01-09      1
1                    1        2018-03-06-17      1
2                    1        2018-03-08-16      1
3                    1        2018-03-11-19      1
4                    1        2018-03-11-20      1
...                ...                  ...    ...
3297                27        2018-05-13-15      1
3298                27        2018-05-23-19      1
3299                28        2018-02-07-17      1
3300                28        2018-03-14-19      1
3301                28        2018-05-12-11      1

[3302 rows x 3 columns]


In [21]:
features_cr_df

,index,year-month-date-hour,hour,start,weekday,12,4,2,10,11,...,26,14,5,7,8,25,15,27,28,21
0,0,2018-02-02-00,0,2018-02-02,4,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2018-02-02-01,1,2018-02-02,4,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2018-02-02-02,2,2018-02-02,4,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,2018-02-02-03,3,2018-02-02,4,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,2018-02-02-04,4,2018-02-02,4,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3619,3619,2018-07-02-19,19,2018-07-02,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3620,3620,2018-07-02-20,20,2018-07-02,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3621,3621,2018-07-02-21,21,2018-07-02,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3622,3622,2018-07-02-22,22,2018-07-02,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
features_cr_df['year-month-date-hour'] = pd.to_datetime(features_cr_df['year-month-date-hour'], format='%Y-%m-%d-%H') # datetime formatting 

In [23]:
features_cr_df.columns

Index(['index', 'year-month-date-hour', 'hour', 'start', 'weekday', '12', '4',
       '2', '10', '11', '6', '19', '1', '9', '18', '3', '16', '13', '20', '17',
       '24', '22', '23', '26', '14', '5', '7', '8', '25', '15', '27', '28',
       '21'],
      dtype='object')

In [24]:
features_cr_df.head()

,index,year-month-date-hour,hour,start,weekday,12,4,2,10,11,...,26,14,5,7,8,25,15,27,28,21
0,0,2018-02-02 00:00:00,0,2018-02-02,4,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2018-02-02 01:00:00,1,2018-02-02,4,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2018-02-02 02:00:00,2,2018-02-02,4,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,2018-02-02 03:00:00,3,2018-02-02,4,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,2018-02-02 04:00:00,4,2018-02-02,4,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### ・ 加速度センサ
次に、加速度センサーの特徴を抽出し、両者を組み合わせます。

**加速度センサーのデータでは、すべてのアクティビティが利用できるわけではありません。seg_label_list.から利用可能なアクティビティを確認することができます。** 

In [25]:
temp=np.array(seg_label_list)
print(np.unique(temp))

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 16 17 18 19 20 22 23 24 25 26
 27]


In [26]:
features_cr_df.columns

Index(['index', 'year-month-date-hour', 'hour', 'start', 'weekday', '12', '4',
       '2', '10', '11', '6', '19', '1', '9', '18', '3', '16', '13', '20', '17',
       '24', '22', '23', '26', '14', '5', '7', '8', '25', '15', '27', '28',
       '21'],
      dtype='object')

In [27]:
from datetime import timedelta

In [28]:
print(len(features_cr_df))

3624


**加速度センサーのデータについては、標準偏差しか取っていません。もっといろいろなことができるはずです。また、シンプルにするために加速度センサーを搭載していないアクティビティは削除しましたが、それらを残して、あなたが考える他の機能値で埋めることも可能です**。

In [29]:
for act in ACT_LABELS:
    features_cr_df['{0}_x_std'.format(act)] = 0
    features_cr_df['{0}_y_std'.format(act)] = 0
    features_cr_df['{0}_z_std'.format(act)] = 0


In [30]:
features_cr_df.head()

,index,year-month-date-hour,hour,start,weekday,12,4,2,10,11,...,15_z_std,27_x_std,27_y_std,27_z_std,28_x_std,28_y_std,28_z_std,21_x_std,21_y_std,21_z_std
0,0,2018-02-02 00:00:00,0,2018-02-02,4,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,1,2018-02-02 01:00:00,1,2018-02-02,4,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,2,2018-02-02 02:00:00,2,2018-02-02,4,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,3,2018-02-02 03:00:00,3,2018-02-02,4,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,4,2018-02-02 04:00:00,4,2018-02-02,4,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


**ここで、分割された各ファイルの標準値を入れ、時間機能と組み合わせて1つのファイルを作成します。**

In [31]:
from time import strptime

for ind in list(features_cr_df.index):
    for i, seg in enumerate(seg_list):
        ct = seg['datetime'].max().strftime('%Y-%m-%d %H:%M:%S')
        ct = datetime.datetime.strptime(ct,'%Y-%m-%d %H:%M:%S')
        st = features_cr_df.loc[ind, 'year-month-date-hour']
        ft = features_cr_df.loc[ind, 'year-month-date-hour'] + datetime.timedelta(minutes=59)
        if (ct>=st) & (ct<=ft):
            for xyz in ['x', 'y', 'z']:
                col = str(seg_label_list[i]) + '_' + xyz + '_' + 'std'
                features_cr_df.loc[ind, col] = seg[xyz].std(ddof=0)

In [32]:
features_cr_df.head()

,index,year-month-date-hour,hour,start,weekday,12,4,2,10,11,...,15_z_std,27_x_std,27_y_std,27_z_std,28_x_std,28_y_std,28_z_std,21_x_std,21_y_std,21_z_std
0,0,2018-02-02 00:00:00,0,2018-02-02,4,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0,0,0,0,0,0
1,1,2018-02-02 01:00:00,1,2018-02-02,4,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0,0,0,0,0,0
2,2,2018-02-02 02:00:00,2,2018-02-02,4,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0,0,0,0,0,0
3,3,2018-02-02 03:00:00,3,2018-02-02,4,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0,0,0,0,0,0
4,4,2018-02-02 04:00:00,4,2018-02-02,4,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0,0,0,0,0,0


In [33]:
features_cr_df.columns

Index(['index', 'year-month-date-hour', 'hour', 'start', 'weekday', '12', '4',
       '2', '10', '11',
       ...
       '15_z_std', '27_x_std', '27_y_std', '27_z_std', '28_x_std', '28_y_std',
       '28_z_std', '21_x_std', '21_y_std', '21_z_std'],
      dtype='object', length=117)

In [34]:
features_cr_df['2_x_std'].value_counts()# just to check the value quantity

0.000000    3577
0.040732       1
0.045085       1
3.483244       1
1.979816       1
0.037273       1
0.039120       1
0.032802       1
0.027086       1
0.034582       1
0.039374       1
0.040464       1
0.586863       1
1.369985       1
2.022897       1
2.340990       1
0.038937       1
0.039358       1
0.660642       1
0.047985       1
0.043156       1
0.039688       1
0.054866       1
9.133890       1
0.034553       1
9.238229       1
0.046257       1
0.038934       1
0.038104       1
1.228533       1
0.040705       1
1.951048       1
0.056287       1
0.020580       1
0.066396       1
0.043699       1
0.403801       1
0.035555       1
0.038289       1
0.033584       1
0.037151       1
0.026558       1
0.492752       1
9.315801       1
9.406627       1
9.034913       1
6.823700       1
0.042568       1
Name: 2_x_std, dtype: int64

## 2.3 学習

In [35]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import classification_report

X = features_cr_df[0:-1]
y = features_cr_df[1:]

In [36]:
# Sort values before data split is very important as we are splitting by length
X = X.sort_values(by="year-month-date-hour")
y = y.sort_values(by="year-month-date-hour")

In [37]:
X.shape, y.shape

((3623, 117), (3623, 117))

In [38]:
# Train Test Split
X_train = X[:int(len(X)*0.7026)]
X_test = X[int(len(X)*0.7026):]
y_train = y[:int(len(y)*0.7026)]
y_test = y[int(len(y)*0.7026):]

In [39]:
display(X.head())
display(y.head())

,index,year-month-date-hour,hour,start,weekday,12,4,2,10,11,...,15_z_std,27_x_std,27_y_std,27_z_std,28_x_std,28_y_std,28_z_std,21_x_std,21_y_std,21_z_std
0,0,2018-02-02 00:00:00,0,2018-02-02,4,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0,0,0,0,0,0
1,1,2018-02-02 01:00:00,1,2018-02-02,4,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0,0,0,0,0,0
2,2,2018-02-02 02:00:00,2,2018-02-02,4,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0,0,0,0,0,0
3,3,2018-02-02 03:00:00,3,2018-02-02,4,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0,0,0,0,0,0
4,4,2018-02-02 04:00:00,4,2018-02-02,4,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0,0,0,0,0,0


,index,year-month-date-hour,hour,start,weekday,12,4,2,10,11,...,15_z_std,27_x_std,27_y_std,27_z_std,28_x_std,28_y_std,28_z_std,21_x_std,21_y_std,21_z_std
1,1,2018-02-02 01:00:00,1,2018-02-02,4,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0,0,0,0,0,0
2,2,2018-02-02 02:00:00,2,2018-02-02,4,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0,0,0,0,0,0
3,3,2018-02-02 03:00:00,3,2018-02-02,4,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0,0,0,0,0,0
4,4,2018-02-02 04:00:00,4,2018-02-02,4,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0,0,0,0,0,0
5,5,2018-02-02 05:00:00,5,2018-02-02,4,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0,0,0,0,0,0


In [40]:
# del X, y

In [41]:
X_train.sort_values(by="year-month-date-hour")
X_test.sort_values(by="year-month-date-hour")

,index,year-month-date-hour,hour,start,weekday,12,4,2,10,11,...,15_z_std,27_x_std,27_y_std,27_z_std,28_x_std,28_y_std,28_z_std,21_x_std,21_y_std,21_z_std
2545,2545,2018-05-19 01:00:00,1,2018-05-19,5,13.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0,0,0,0,0,0
2546,2546,2018-05-19 02:00:00,2,2018-05-19,5,1.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0,0,0,0,0,0
2547,2547,2018-05-19 03:00:00,3,2018-05-19,5,15.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0,0,0,0,0,0
2548,2548,2018-05-19 04:00:00,4,2018-05-19,5,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0,0,0,0,0,0
2549,2549,2018-05-19 05:00:00,5,2018-05-19,5,15.0,3.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3618,3618,2018-07-02 18:00:00,18,2018-07-02,0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0,0,0,0,0,0
3619,3619,2018-07-02 19:00:00,19,2018-07-02,0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0,0,0,0,0,0
3620,3620,2018-07-02 20:00:00,20,2018-07-02,0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0,0,0,0,0,0
3621,3621,2018-07-02 21:00:00,21,2018-07-02,0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0,0,0,0,0,0


In [42]:
y_train.sort_values(by="year-month-date-hour")
y_test.sort_values(by="year-month-date-hour")

,index,year-month-date-hour,hour,start,weekday,12,4,2,10,11,...,15_z_std,27_x_std,27_y_std,27_z_std,28_x_std,28_y_std,28_z_std,21_x_std,21_y_std,21_z_std
2546,2546,2018-05-19 02:00:00,2,2018-05-19,5,1.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0,0,0,0,0,0
2547,2547,2018-05-19 03:00:00,3,2018-05-19,5,15.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0,0,0,0,0,0
2548,2548,2018-05-19 04:00:00,4,2018-05-19,5,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0,0,0,0,0,0
2549,2549,2018-05-19 05:00:00,5,2018-05-19,5,15.0,3.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0,0,0,0,0,0
2550,2550,2018-05-19 06:00:00,6,2018-05-19,5,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3619,3619,2018-07-02 19:00:00,19,2018-07-02,0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0,0,0,0,0,0
3620,3620,2018-07-02 20:00:00,20,2018-07-02,0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0,0,0,0,0,0
3621,3621,2018-07-02 21:00:00,21,2018-07-02,0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0,0,0,0,0,0
3622,3622,2018-07-02 22:00:00,22,2018-07-02,0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0,0,0,0,0,0


In [43]:
# keep the year-month-date-hour before start dropping the columns as we will need it to make the submission file
y_time = y_test['year-month-date-hour']

In [44]:
cols=['year-month-date-hour', 'hour', 'start', 'weekday']
X_train = X_train.drop(['year-month-date-hour', 'start'], axis=1)
y_train = y_train.drop(cols, axis=1)
X_test = X_test.drop(['year-month-date-hour', 'start'], axis=1)
y_test = y_test.drop(cols, axis=1)

In [45]:
y_train=y_train.drop(['index', '1_x_std', '2_x_std', '4_x_std',
       '9_x_std', '10_x_std', '11_x_std', '14_x_std', '19_x_std', '1_y_std',
       '2_y_std', '4_y_std', '9_y_std', '10_y_std', '11_y_std', '14_y_std',
       '19_y_std', '1_z_std', '2_z_std', '4_z_std', '9_z_std', '10_z_std',
       '11_z_std', '14_z_std', '19_z_std'], axis=1)

y_test=y_test.drop(['index', '1_x_std', '2_x_std', '4_x_std',
       '9_x_std', '10_x_std', '11_x_std', '14_x_std', '19_x_std', '1_y_std',
       '2_y_std', '4_y_std', '9_y_std', '10_y_std', '11_y_std', '14_y_std',
       '19_y_std', '1_z_std', '2_z_std', '4_z_std', '9_z_std', '10_z_std',
       '11_z_std', '14_z_std', '19_z_std'], axis=1)


In [46]:
y_train[y_train < 1] = 0
y_train[y_train >= 1] = 1
y_test[y_test < 1] = 0
y_test[y_test >= 1] = 1

In [47]:
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

## 2.4 評価

In [48]:
y_test

,12,4,2,10,11,6,19,1,9,18,...,15_z_std,27_x_std,27_y_std,27_z_std,28_x_std,28_y_std,28_z_std,21_x_std,21_y_std,21_z_std
2546,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0,0,0,0,0,0
2547,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0,0,0,0,0,0
2548,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0,0.0,0.0,0.0,0,0,0,0,0,0
2549,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0.0,0.0,0.0,0,0,0,0,0,0
2550,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0.0,0.0,0.0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3619,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0,0,0,0,0,0
3620,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0,0,0,0,0,0
3621,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0,0,0,0,0,0
3622,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0,0,0,0,0,0


In [49]:
y_pred.shape

(1078, 88)

In [50]:
from sklearn.metrics import classification_report, accuracy_score
print('accuracy: {0} %'.format(100*accuracy_score(y_test.iloc[:, :28], y_pred[:, :28])))
print(classification_report(y_test.iloc[:, :28], y_pred[:, :28]))

accuracy: 36.178107606679035 %
              precision    recall  f1-score   support

           0       0.67      0.64      0.65       272
           1       0.46      0.70      0.56       287
           2       0.37      0.36      0.36       143
           3       0.00      0.00      0.00        59
           4       0.30      0.16      0.21        90
           5       1.00      0.12      0.21        43
           6       0.17      0.02      0.04        92
           7       0.20      0.03      0.05        68
           8       0.45      0.10      0.16        52
           9       0.00      0.00      0.00        39
          10       0.09      0.03      0.04        34
          11       0.00      0.00      0.00        29
          12       0.00      0.00      0.00        37
          13       0.00      0.00      0.00        28
          14       0.00      0.00      0.00         1
          15       0.00      0.00      0.00        14
          16       0.00      0.00      0.00       

/home/riku-shinohara/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/riku-shinohara/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/riku-shinohara/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/riku-shinohara/.lo

In [51]:
from sklearn.metrics import classification_report, accuracy_score
print('accuracy: {0} %'.format(100*accuracy_score(y_test, y_pred)))
print(classification_report(y_test, y_pred))

accuracy: 35.89981447124304 %
              precision    recall  f1-score   support

           0       0.67      0.64      0.65       272
           1       0.46      0.70      0.56       287
           2       0.37      0.36      0.36       143
           3       0.00      0.00      0.00        59
           4       0.30      0.16      0.21        90
           5       1.00      0.12      0.21        43
           6       0.17      0.02      0.04        92
           7       0.20      0.03      0.05        68
           8       0.45      0.10      0.16        52
           9       0.00      0.00      0.00        39
          10       0.09      0.03      0.04        34
          11       0.00      0.00      0.00        29
          12       0.00      0.00      0.00        37
          13       0.00      0.00      0.00        28
          14       0.00      0.00      0.00         1
          15       0.00      0.00      0.00        14
          16       0.00      0.00      0.00        

/home/riku-shinohara/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/riku-shinohara/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/riku-shinohara/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/riku-shinohara/.lo

# 3.Prediction (with Care Record Only)
Hourly Based Activity Forecasting with Care Records Only

**We are going to put the data frames to different data frames for ease of use. It is not mandatory to do so**

In [52]:
act_df = act_df.drop_duplicates('id') # to drop duplicate columns
act_df = act_df.dropna()# to drop the nan values

## 3.1 Preprocessing

**Now we need to format the time stamp**

In [53]:
# Convert [start/finish] to datetime
act_df['start'] = pd.to_datetime(act_df['start'], format='%Y-%m-%d %H:%M:%S %z')
act_df['finish'] = pd.to_datetime(act_df['finish'], format='%Y-%m-%d %H:%M:%S %z')

In [54]:
act_df.shape

(9543, 11)

In [55]:
act_df.head()

,id,user_id,activity_type_id,activity_type,target_id,activity2user_id,start,finish,year-month-date-hour,year-month-date,hour
3371,179,14,18,申し送り・ミーティング,3,14624,2018-02-02 09:47:16+09:00,2018-02-02 09:47:21+09:00,2018-02-02-09,2018-02-02,9
3294,187,14,10,日中利用者対応,35,12354,2018-02-02 11:12:03+09:00,2018-02-02 11:09:40+09:00,2018-02-02-11,2018-02-02,11
711,189,14,17,手書き記録,30,11802,2018-02-02 11:27:59+09:00,2018-02-02 11:28:00+09:00,2018-02-02-11,2018-02-02,11
817,192,14,2,食事・服薬,30,11179,2018-02-02 11:56:46+09:00,2018-02-02 12:16:47+09:00,2018-02-02-11,2018-02-02,11
823,193,14,2,食事・服薬,32,11855,2018-02-02 12:14:54+09:00,2018-02-02 12:14:55+09:00,2018-02-02-12,2018-02-02,12


In [56]:
act_df['year-month-date'] = act_df['start'].dt.date.astype(str) # to create year-month-date column
act_df['hour'] = act_df['start'].dt.hour # to create hour column
act_df['year-month-date-hour'] = act_df['year-month-date'].astype(str) + '-' +act_df['hour'].astype(str).str.zfill(2) # to create year-month-date-hour column

act_df = act_df.sort_values('start')
ACT_LABELS = list(act_df['activity_type_id'].value_counts().index) # 1)Activity Type List
USERS = list(act_df['user_id'].value_counts().index) # 2)User ID List

In [57]:
display(act_df.head())
print(ACT_LABELS) # to see which activities are done by this user.

,id,user_id,activity_type_id,activity_type,target_id,activity2user_id,start,finish,year-month-date-hour,year-month-date,hour
3371,179,14,18,申し送り・ミーティング,3,14624,2018-02-02 09:47:16+09:00,2018-02-02 09:47:21+09:00,2018-02-02-09,2018-02-02,9
3294,187,14,10,日中利用者対応,35,12354,2018-02-02 11:12:03+09:00,2018-02-02 11:09:40+09:00,2018-02-02-11,2018-02-02,11
711,189,14,17,手書き記録,30,11802,2018-02-02 11:27:59+09:00,2018-02-02 11:28:00+09:00,2018-02-02-11,2018-02-02,11
817,192,14,2,食事・服薬,30,11179,2018-02-02 11:56:46+09:00,2018-02-02 12:16:47+09:00,2018-02-02-11,2018-02-02,11
823,193,14,2,食事・服薬,32,11855,2018-02-02 12:14:54+09:00,2018-02-02 12:14:55+09:00,2018-02-02-12,2018-02-02,12


[12, 4, 2, 10, 11, 6, 19, 1, 9, 18, 3, 16, 13, 20, 17, 24, 22, 23, 26, 14, 5, 7, 8, 25, 15, 27, 28, 21]


## 3.2 Feature Extraction 

In [58]:
date_start = act_df['start'].min()
date_finish = act_df['start'].max()

In [59]:
df_y

,year-month-date-hour,hour,start,weekday
0,2018-02-02-00,0,2018-02-02,4
1,2018-02-02-01,1,2018-02-02,4
2,2018-02-02-02,2,2018-02-02,4
3,2018-02-02-03,3,2018-02-02,4
4,2018-02-02-04,4,2018-02-02,4
...,...,...,...,...
3619,2018-07-02-19,19,2018-07-02,0
3620,2018-07-02-20,20,2018-07-02,0
3621,2018-07-02-21,21,2018-07-02,0
3622,2018-07-02-22,22,2018-07-02,0


In [60]:
import datetime

time = date_start.date()
time_list = []
while time <= date_finish:
    for hour in range(0, 24):
        year_month_date_hour = str(time) + '-' +str(hour).zfill(2)
        print(year_month_date_hour)
        time_list.append([year_month_date_hour, hour, time, time.weekday()])
    time = time + datetime.timedelta(days=1)

feat_cols=['year-month-date-hour', 'hour', 'start', 'weekday']
df_y = pd.DataFrame(time_list, columns=feat_cols)

2018-02-02-00
2018-02-02-01
2018-02-02-02
2018-02-02-03
2018-02-02-04
2018-02-02-05
2018-02-02-06
2018-02-02-07
2018-02-02-08
2018-02-02-09
2018-02-02-10
2018-02-02-11
2018-02-02-12
2018-02-02-13
2018-02-02-14
2018-02-02-15
2018-02-02-16
2018-02-02-17
2018-02-02-18
2018-02-02-19
2018-02-02-20
2018-02-02-21
2018-02-02-22
2018-02-02-23
2018-02-03-00
2018-02-03-01
2018-02-03-02
2018-02-03-03
2018-02-03-04
2018-02-03-05
2018-02-03-06
2018-02-03-07
2018-02-03-08
2018-02-03-09
2018-02-03-10
2018-02-03-11
2018-02-03-12
2018-02-03-13
2018-02-03-14
2018-02-03-15
2018-02-03-16
2018-02-03-17
2018-02-03-18
2018-02-03-19
2018-02-03-20
2018-02-03-21
2018-02-03-22
2018-02-03-23
2018-02-04-00
2018-02-04-01
2018-02-04-02
2018-02-04-03
2018-02-04-04
2018-02-04-05
2018-02-04-06
2018-02-04-07
2018-02-04-08
2018-02-04-09
2018-02-04-10
2018-02-04-11
2018-02-04-12
2018-02-04-13
2018-02-04-14
2018-02-04-15
2018-02-04-16
2018-02-04-17
2018-02-04-18
2018-02-04-19
2018-02-04-20
2018-02-04-21
2018-02-04-22
2018-0

/tmp/ipykernel_740173/3771523147.py:5: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  while time <= date_finish:


In [61]:
date_start.date()

datetime.date(2018, 2, 2)

In [62]:
df = act_df.copy() # make a copy so that if needed the main data frame can still be utilised

In [63]:
df.groupby(["activity_type_id", "year-month-date-hour"], as_index=False).count()

,activity_type_id,year-month-date-hour,id,user_id,activity_type,target_id,activity2user_id,start,finish,year-month-date,hour
0,1,2018-03-01-09,1,1,1,1,1,1,1,1,1
1,1,2018-03-06-17,1,1,1,1,1,1,1,1,1
2,1,2018-03-08-16,1,1,1,1,1,1,1,1,1
3,1,2018-03-11-19,1,1,1,1,1,1,1,1,1
4,1,2018-03-11-20,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...
3297,27,2018-05-13-15,1,1,1,1,1,1,1,1,1
3298,27,2018-05-23-19,1,1,1,1,1,1,1,1,1
3299,28,2018-02-07-17,1,1,1,1,1,1,1,1,1
3300,28,2018-03-14-19,1,1,1,1,1,1,1,1,1


In [64]:
def makeFeatures(df):
    feat = df.groupby(['activity_type_id', 'year-month-date-hour'], as_index=False).count()[['activity_type_id', 'year-month-date-hour','id']].rename(columns={'id':'count'})
    result_df = df_y.copy()
    for act in ACT_LABELS:
        result_df = pd.merge(result_df, feat[feat['activity_type_id'] == act][['year-month-date-hour', 'count']], on='year-month-date-hour', how="left").rename(columns={'count':str(act)})
    return result_df

tmp_df = df
features_cr_df = makeFeatures(tmp_df)
features_cr_df = features_cr_df.fillna(0).sort_values('year-month-date-hour')
features_cr_df = features_cr_df.reset_index()

In [65]:
features_cr_df

,index,year-month-date-hour,hour,start,weekday,12,4,2,10,11,...,26,14,5,7,8,25,15,27,28,21
0,0,2018-02-02-00,0,2018-02-02,4,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2018-02-02-01,1,2018-02-02,4,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2018-02-02-02,2,2018-02-02,4,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,2018-02-02-03,3,2018-02-02,4,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,2018-02-02-04,4,2018-02-02,4,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3619,3619,2018-07-02-19,19,2018-07-02,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3620,3620,2018-07-02-20,20,2018-07-02,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3621,3621,2018-07-02-21,21,2018-07-02,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3622,3622,2018-07-02-22,22,2018-07-02,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 3.3 Classification 

In [66]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import classification_report

X = features_cr_df[0:-1]
y = features_cr_df[1:]

In [67]:
# Sort values before data split is very important as we are splitting by length
X = X.sort_values(by="year-month-date-hour")
y = y.sort_values(by="year-month-date-hour")

In [68]:
# train test split
X_train = X[:int(len(X)*0.7026)]
X_test = X[int(len(X)*0.7026):]
y_train = y[:int(len(y)*0.7026)]
y_test = y[int(len(y)*0.7026):]

In [69]:
X_train.sort_values(by="year-month-date-hour")

,index,year-month-date-hour,hour,start,weekday,12,4,2,10,11,...,26,14,5,7,8,25,15,27,28,21
0,0,2018-02-02-00,0,2018-02-02,4,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2018-02-02-01,1,2018-02-02,4,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2018-02-02-02,2,2018-02-02,4,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,2018-02-02-03,3,2018-02-02,4,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,2018-02-02-04,4,2018-02-02,4,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2540,2540,2018-05-18-20,20,2018-05-18,4,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2541,2541,2018-05-18-21,21,2018-05-18,4,11.0,4.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2542,2542,2018-05-18-22,22,2018-05-18,4,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2543,2543,2018-05-18-23,23,2018-05-18,4,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [70]:
X_test.sort_values(by="year-month-date-hour")

,index,year-month-date-hour,hour,start,weekday,12,4,2,10,11,...,26,14,5,7,8,25,15,27,28,21
2545,2545,2018-05-19-01,1,2018-05-19,5,13.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2546,2546,2018-05-19-02,2,2018-05-19,5,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2547,2547,2018-05-19-03,3,2018-05-19,5,15.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2548,2548,2018-05-19-04,4,2018-05-19,5,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2549,2549,2018-05-19-05,5,2018-05-19,5,15.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3618,3618,2018-07-02-18,18,2018-07-02,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3619,3619,2018-07-02-19,19,2018-07-02,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3620,3620,2018-07-02-20,20,2018-07-02,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3621,3621,2018-07-02-21,21,2018-07-02,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [71]:
y_train.sort_values(by="year-month-date-hour")

,index,year-month-date-hour,hour,start,weekday,12,4,2,10,11,...,26,14,5,7,8,25,15,27,28,21
1,1,2018-02-02-01,1,2018-02-02,4,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2018-02-02-02,2,2018-02-02,4,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,2018-02-02-03,3,2018-02-02,4,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,2018-02-02-04,4,2018-02-02,4,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,5,2018-02-02-05,5,2018-02-02,4,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2541,2541,2018-05-18-21,21,2018-05-18,4,11.0,4.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2542,2542,2018-05-18-22,22,2018-05-18,4,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2543,2543,2018-05-18-23,23,2018-05-18,4,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2544,2544,2018-05-19-00,0,2018-05-19,5,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [72]:
y_test.sort_values(by="year-month-date-hour")

,index,year-month-date-hour,hour,start,weekday,12,4,2,10,11,...,26,14,5,7,8,25,15,27,28,21
2546,2546,2018-05-19-02,2,2018-05-19,5,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2547,2547,2018-05-19-03,3,2018-05-19,5,15.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2548,2548,2018-05-19-04,4,2018-05-19,5,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2549,2549,2018-05-19-05,5,2018-05-19,5,15.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2550,2550,2018-05-19-06,6,2018-05-19,5,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3619,3619,2018-07-02-19,19,2018-07-02,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3620,3620,2018-07-02-20,20,2018-07-02,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3621,3621,2018-07-02-21,21,2018-07-02,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3622,3622,2018-07-02-22,22,2018-07-02,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [73]:
# keep the year-month-date-hour before start dropping the columns as we will need it to make the submission file
y_time = y_test['year-month-date-hour']

In [74]:
X_train = X_train.drop(['year-month-date-hour', 'start'], axis=1)
y_train = y_train.drop(feat_cols, axis=1)
X_test = X_test.drop(['year-month-date-hour', 'start'], axis=1)
y_test = y_test.drop(feat_cols, axis=1)

In [75]:
y_train=y_train.drop(columns=['index'])
y_test=y_test.drop(columns=['index'])

In [76]:
y_test

,12,4,2,10,11,6,19,1,9,18,...,26,14,5,7,8,25,15,27,28,21
2546,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2547,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2548,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2549,15.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2550,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3619,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3620,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3621,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3622,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [77]:
y_train[y_train < 1] = 0
y_train[y_train >= 1] = 1
y_test[y_test < 1] = 0
y_test[y_test >= 1] = 1

In [85]:
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [86]:
y_pred

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

## 3.4 Evaluation

In [94]:
from sklearn.metrics import classification_report, accuracy_score
print('accuracy: {0} %'.format(100*accuracy_score(y_test, y_pred)))
report_df = classification_report(y_test, y_pred, output_dict=True)
report = pd.DataFrame(report_df).T
display(report)

accuracy: 35.25046382189239 %


/home/riku-shinohara/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/riku-shinohara/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/riku-shinohara/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/riku-shinohara/.lo

,precision,recall,f1-score,support
0,0.639344,0.716912,0.675910,272.0
1,0.426778,0.710801,0.533333,287.0
2,0.500000,0.391608,0.439216,143.0
3,0.500000,0.016949,0.032787,59.0
4,0.397590,0.366667,0.381503,90.0
5,0.833333,0.232558,0.363636,43.0
6,0.333333,0.021739,0.040816,92.0
7,0.454545,0.073529,0.126582,68.0
8,0.466667,0.134615,0.208955,52.0
9,0.000000,0.000000,0.000000,39.0


In [95]:
import dataframe_image as dfi
report_df = pd.DataFrame(report_df).T
report_df.dfi.export("reporta.png")

[0727/095617.693374:ERROR:bus.cc(398)] Failed to connect to the bus: Failed to connect to socket /var/run/dbus/system_bus_socket: No such file or directory
[0727/095617.693555:ERROR:bus.cc(398)] Failed to connect to the bus: Failed to connect to socket /var/run/dbus/system_bus_socket: No such file or directory
[0727/095617.694764:WARNING:bluez_dbus_manager.cc(247)] Floss manager not present, cannot set Floss enable/disable.
[0727/095617.707495:WARNING:sandbox_linux.cc(376)] InitializeSandbox() called with multiple threads in process gpu-process.
[0727/095617.831074:INFO:headless_shell.cc(660)] Written to file /tmp/tmpxt5jcn1g/temp.png.
[0727/095618.035473:ERROR:bus.cc(398)] Failed to connect to the bus: Failed to connect to socket /var/run/dbus/system_bus_socket: No such file or directory
[0727/095618.035549:ERROR:bus.cc(398)] Failed to connect to the bus: Failed to connect to socket /var/run/dbus/system_bus_socket: No such file or directory
[0727/095618.037003:WARNING:bluez_dbus_manag

# 4.Generate submission file

In [81]:
y_pred = pd.DataFrame(y_pred,columns=y_test.columns)

In [82]:
y_pred.insert(0, y_time.name,y_time.to_numpy())

In [83]:
y_pred.head()

,year-month-date-hour,12,4,2,10,11,6,19,1,9,...,26,14,5,7,8,25,15,27,28,21
0,2018-05-19-02,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2018-05-19-03,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2018-05-19-04,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2018-05-19-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2018-05-19-06,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**Convert this file to csv and submit!**